In [6]:
import gym
import numpy as np
#导入库

env = gym.make("MountainCar-v0")
#环境为gym中的MountainCar-v0

# Q-Learing的参数设置
LEARNING_RATE = 0.1
DISCOUNT = 0.95
EPISODES = 500

SHOW_EVERY = 1000

# 探索设置
epsilon = 1  # 不是常数
START_EPSILON_DECAYING = 1
END_EPSILON_DECAYING = EPISODES//2
epsilon_decay_value = epsilon/(END_EPSILON_DECAYING - START_EPSILON_DECAYING)

DISCRETE_OS_SIZE = [20, 20]
discrete_os_win_size = (env.observation_space.high - env.observation_space.low) / DISCRETE_OS_SIZE

print(discrete_os_win_size)


def get_discrete_state(state):
    discrete_state = (state[1] - env.observation_space.low)/discrete_os_win_size
    return tuple(discrete_state.astype(np.int64))  # 我们使用这个元组来查找Q中可用操作的3个Q值


q_table = np.random.uniform(low=-2, high=0, size=(DISCRETE_OS_SIZE + [env.action_space.n]))


for episode in range(EPISODES):
    state = env.reset()
    print(state)
    discrete_state = get_discrete_state(state[0])

    if episode % SHOW_EVERY == 0:
        render = True
        print(episode)
    else:
        render = False

    done = False
    while not done:
        if np.random.random() > epsilon:
            #根据Q-table中的数据做出行为
            action = np.argmax(q_table[discrete_state])
        else:
            #随机行为
            action = np.random.randint(0, env.action_space.n)

        new_state, reward, done, __, _ = env.step(action)
        new_discrete_state = get_discrete_state(new_state)

        #如果模拟在最后一步之后仍未结束，则更新Q表
        if not done:
            # 下一步可能的最大Q值（针对新状态）
            max_future_q = np.max(q_table[new_discrete_state])
            # 当前的Q值（用于当前状态和执行的动作）
            current_q = q_table[discrete_state + (action,)]
            # 对当前状态和动作的新Q值的方程
            new_q = (1 - LEARNING_RATE) * current_q + LEARNING_RATE * (reward + DISCOUNT * max_future_q)
            # 根据新的q值更新Q-table
            q_table[discrete_state + (action,)] = new_q
            # 模拟结束（对于任何原因）-如果达到目标位置-直接用奖励更新Q值

        elif new_state[0] >= env.goal_position:
            # q_table[discrete_state + (action,)] = reward
            q_table[discrete_state + (action,)] = 0
            print("we made it on episode {}".format(episode))

        discrete_state = new_discrete_state

        if render:
            env.render()

    # 如果集数在衰减范围内，则每个episode都会进行衰减
    if END_EPSILON_DECAYING >= episode >= START_EPSILON_DECAYING:
        epsilon -= epsilon_decay_value

np.save("q_table.npy", arr=q_table)
print(q_table)

env.close()


[0.09  0.007]
(array([-0.58851135,  0.        ], dtype=float32), {})
0
we made it on episode 0
(array([-0.47469482,  0.        ], dtype=float32), {})
we made it on episode 1
(array([-0.59621125,  0.        ], dtype=float32), {})
we made it on episode 2
(array([-0.5995497,  0.       ], dtype=float32), {})
we made it on episode 3
(array([-0.41767558,  0.        ], dtype=float32), {})
we made it on episode 4
(array([-0.40028554,  0.        ], dtype=float32), {})
we made it on episode 5
(array([-0.5813821,  0.       ], dtype=float32), {})
we made it on episode 6
(array([-0.51829803,  0.        ], dtype=float32), {})
we made it on episode 7
(array([-0.49042884,  0.        ], dtype=float32), {})
we made it on episode 8
(array([-0.4005619,  0.       ], dtype=float32), {})
we made it on episode 9
(array([-0.5943653,  0.       ], dtype=float32), {})
we made it on episode 10
(array([-0.49519402,  0.        ], dtype=float32), {})
we made it on episode 11
(array([-0.42310116,  0.        ], dtype=f

In [11]:
import gym
import numpy as np


env = gym.make("MountainCar-v0")

LEARNING_RATE = 0.1
DISCOUNT = 0.95
EPISODES = 10

DISCRETE_OS_SIZE = [20, 20]
discrete_os_win_size = (env.observation_space.high - env.observation_space.low) / DISCRETE_OS_SIZE

def get_discrete_state(state):
    discrete_state = (state[1] - env.observation_space.low)/discrete_os_win_size
    return tuple(discrete_state.astype(np.int64))  

q_table = np.load(file="q_table.npy")

for episode in range(EPISODES):
    state = env.reset()
    discrete_state = get_discrete_state(state[0])

    rewards = 0
    done = False
    #print(done)
    while not done:
        action = np.argmax(q_table[discrete_state])
        new_state, reward, done, __, _ = env.step(action)
        new_discrete_state = get_discrete_state(new_state)

        rewards += reward

        if done and new_state[0] >= env.goal_position:
            print("we made it on episode {}, rewards {}".format(episode, rewards))

        discrete_state = new_discrete_state
        env.render()

env.close()

we made it on episode 0, rewards -315.0
we made it on episode 1, rewards -311.0
we made it on episode 2, rewards -224.0
we made it on episode 3, rewards -314.0
we made it on episode 4, rewards -311.0
we made it on episode 5, rewards -314.0
we made it on episode 6, rewards -223.0
we made it on episode 7, rewards -223.0
we made it on episode 8, rewards -315.0
we made it on episode 9, rewards -221.0
